In [ ]:
import numpy as np
import pandas as pd
import dgutils.pandas as dgp
import plotly.express as px
import plotly.figure_factory as ff
from functools import reduce
from model_utils.utils_model import Evaluator, DataEncoder

In [ ]:
import model_utils.utils_model as us1

In [ ]:
import torch
from scipy.special import softmax

In [ ]:
def stem2db_str(df_stem, seq_len):
    db_str = ['.'] * seq_len
    for _, row in df_stem.iterrows():
        bb_x = row['bb_x']
        bb_y = row['bb_y']
        siz = row['siz_x']
        siz_y = row['siz_y']
        assert siz == siz_y
        for i in range(siz):
            db_str[bb_x+i] = '('
            db_str[bb_y-i] = ')'
    return ''.join(db_str)

In [ ]:
predictor_s1 = us1.Predictor('v1.0')

In [ ]:
df = pd.read_pickle('../2021_01_12/data/synthetic_s1_pred_1000_t0p1_k1.pkl.gz')

In [ ]:
df.columns

In [ ]:
evaluator = Evaluator(predictor=None) 

In [ ]:
# take stem for example


def compute_stem_sensitivity(bounding_boxes, stems):
    df_target_stem, df_target_iloop, df_target_hloop = evaluator.make_target_bb_df(bounding_boxes, 
                                                                                   convert_tl_to_tr=True)
    df_stem = pd.DataFrame(stems)
    m = evaluator.calculate_bb_metrics(df_target=df_target_stem, df_pred=df_stem[['bb_x', 'bb_y', 'siz_x', 'siz_y']])
    s = float(m['n_target_identical'])/m['n_target_total']
    return s


df = dgp.add_column(df, 'stem_bb_identical_sensitivity',
                   ['bounding_boxes', 'bb_stem'], compute_stem_sensitivity)

In [ ]:
df_missed = df[df['stem_bb_identical_sensitivity'] < 1.0]

In [ ]:
print(len(df), len(df_missed))

In [ ]:
# row = df_missed.iloc[0]

# row = df_missed.iloc[1]

row = df_missed.iloc[2]


In [ ]:
row

In [ ]:
seq = row['seq']
df_target_stem, df_target_iloop, df_target_hloop = evaluator.make_target_bb_df(row['bounding_boxes'], 
                                                                                convert_tl_to_tr=True)
df_pred_stem = pd.DataFrame(row['bb_stem'])

In [ ]:
seq

In [ ]:
df_target_stem

In [ ]:
df_stem_hit = pd.merge(df_target_stem, df_pred_stem,
                      on=['bb_x', 'bb_y', 'siz_x', 'siz_y'],
                      how='left')

In [ ]:
df_stem_hit

In [ ]:
df_pred_stem

In [ ]:
print(seq)
print(stem2db_str(df_target_stem, len(seq)))

In [ ]:
# for pixels within this target stem bb
# find the probabilities

In [ ]:
# investigate bb:  (inclusive)  [61-64]-[68-71]

In [ ]:
# uniq_stem, uniq_iloop, uniq_hloop = predictor_s1.predict_bb(seq, threshold=0.1, topk=1, perc_cutoff=0)
de = DataEncoder(seq)
yp = predictor_s1.model(torch.tensor(de.x_torch))
yp = {k: v.detach().cpu().numpy()[0, :, :, :] for k, v in yp.items()}


In [ ]:
yp.keys()

In [ ]:
# sequence segment
print(seq[20:24])
print(seq[57:61])

In [ ]:
yp['stem_on'].shape

In [ ]:
yp['stem_on'][0, 20:24, 57:61]

In [ ]:
yp['stem_location_x'].shape

In [ ]:
pixel_x = 22
pixel_y = 58

In [ ]:
# tr
softmax(yp['stem_location_x'][:, pixel_x, pixel_y])

In [ ]:
softmax(yp['stem_location_y'][:, pixel_x, pixel_y])

In [ ]:
softmax(yp['stem_sm_size'][:, pixel_x, pixel_y])

In [ ]:
yp['stem_sl_size'][0, pixel_x, pixel_y]